In [1]:
import keras

In [2]:
from keras.models import load_model

In [4]:
model_new = load_model("model_saved/Yaw_model.h5")

In [28]:
import coremltools

In [10]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [12]:
coreml_model = coremltools.convert("model_saved/Yaw_model.h5")

# coreml_model.visualize_spec()

coreml_model.save("model_saved/Yaw_prediction.mlmodel")

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 194/194 [00:27<00:00,  7.18 ops/s]


In [51]:
import torch
model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model_yolov5.eval()

TypeError: load() missing 1 required positional argument: 'model'

In [52]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.mlmodel') 

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)



requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


Loading yolov5s.mlmodel for CoreML inference...


Exception: [Errno 2] No such file or directory: 'yolov5s.mlmodel'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

In [57]:
import coremltools as ct

# Load the ONNX model coremltools.convert
onnx_model = ct.convert('yolov5s.pt')

# Convert the ONNX model to a Core ML model
mlmodel = ct.convert(onnx_model)

ValueError: Unable to determine the type of the model, i.e. the source framework. Please provide the value of argument "source", from one of ["tensorflow", "pytorch", "milinternal"]. Note that model conversion requires the source package that generates the model. Please make sure you have the appropriate version of source package installed. E.g., if you're converting model originally trained with TensorFlow 1.14, make sure you have `tensorflow==1.14` installed.

In [ ]:
mlmodel.save('my_mlmodel.mlmodel')


In [ ]:
model_yolov5.to(torch.device('cpu'))

In [44]:
next(model_yolov5.parameters()).is_cuda

False

In [47]:
import onnx

In [45]:
input_names = ['image']
output_names = ['output']

# Export PyTorch model to ONNX
torch.onnx.export(model_yolov5, torch.randn((1, 3, 640, 640)), 'yolov5s.onnx', export_params=True, input_names=input_names, output_names=output_names)

C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:514: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master\models\yolo.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [49]:
import coremltools as ct

# Load the ONNX model
# onnx_model = onnx.load('yolov5s.onnx')

# Convert ONNX model to Core ML model
mlmodel = ct.convert('yolov5s.onnx')

# Save the Core ML model
mlmodel.save('yolov5s.mlmodel')

ValueError: Unable to determine the type of the model, i.e. the source framework. Please provide the value of argument "source", from one of ["tensorflow", "pytorch", "milinternal"]. Note that model conversion requires the source package that generates the model. Please make sure you have the appropriate version of source package installed. E.g., if you're converting model originally trained with TensorFlow 1.14, make sure you have `tensorflow==1.14` installed.

In [26]:
import cv2

# im = cv2.imread("sh/IMG_3695.jpg")
im =  torch.rand(1,700,700,3) 

In [34]:
coremltools.convert( )

ValueError: Expected argument for pytorch "inputs" not provided

In [27]:
traced_model = torch.jit.trace(model_yolov5 , im)

RuntimeError: Given groups=1, weight of size [32, 3, 6, 6], expected input[1, 700, 700, 3] to have 3 channels, but got 700 channels instead